In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!nvidia-smi

Fri Aug 18 07:55:30 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

/content


## Install YOLOv8

⚠️ YOLOv8 is still under heavy development. Breaking changes are being introduced almost weekly. We strive to make our YOLOv8 notebooks work with the latest version of the library. Last tests took place on **27.01.2023** with version **YOLOv8.0.20**.

If you notice that our notebook behaves incorrectly - especially if you experience errors that prevent you from going through the tutorial - don't hesitate! Let us know and open an [issue](https://github.com/roboflow/notebooks/issues) on the Roboflow Notebooks repository.

YOLOv8 can be installed in two ways - from the source and via pip. This is because it is the first iteration of YOLO to have an official package.

In [ ]:
# Pip install method (recommended)

!pip install ultralytics==8.0.20

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.20 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 26.3/78.2 GB disk)


In [ ]:
# Git clone method (for development)

%cd {HOME}
!git clone https://github.com/ElNino9495/depth_estimation_yolo_v8.git
%cd {HOME}/ultralytics
!pip install -e .

from IPython import display
display.clear_output()



## Custom Training

In [ ]:
%cd {HOME}

!yolo task=detect mode=train model=yolov8s.pt data={dataset.location}/data.yaml epochs=50 imgsz=800 plots=True

/content
 27% 5.79M/21.5M [00:00<00:00, 17.5MB/s]
ERROR: Downloaded file 'yolov8s.pt' does not exist or size is < min_bytes=100000.0
yolov8s.pt missing, try downloading from https://github.com/ultralytics/assets/releases/v0.0.0

Traceback (most recent call last):
  File "/usr/local/bin/yolo", line 8, in <module>
    sys.exit(entrypoint())
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/yolo/cfg/__init__.py", line 230, in entrypoint
    model = YOLO(model)
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/yolo/engine/model.py", line 60, in __init__
    {'.pt': self._load, '.yaml': self._new}[suffix](model)
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/yolo/engine/model.py", line 90, in _load
    self.model, self.ckpt = attempt_load_one_weight(weights)
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/nn/tasks.py", line 384, in attempt_load_one_weight
    ckpt = torch_safe_load(weight)  # load ckpt
  File "/usr/local/lib/python3.10/dist-packa

In [ ]:
!ls {HOME}/runs/detect/train/

args.yaml					    train_batch2.jpg
confusion_matrix.png				    train_batch4760.jpg
events.out.tfevents.1687849397.075c838c728e.1856.0  train_batch4761.jpg
F1_curve.png					    train_batch4762.jpg
P_curve.png					    val_batch0_labels.jpg
PR_curve.png					    val_batch0_pred.jpg
R_curve.png					    val_batch1_labels.jpg
results.csv					    val_batch1_pred.jpg
results.png					    val_batch2_labels.jpg
train_batch0.jpg				    val_batch2_pred.jpg
train_batch1.jpg				    weights


In [ ]:
import glob
from IPython.display import Image, display

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/detect/train/confusion_matrix.png', width=600)

/content


FileNotFoundError: ignored

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/detect/train/results.png', width=600)

/content


FileNotFoundError: ignored

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/detect/train/val_batch0_pred.jpg', width=600)

/content


TypeError: ignored

## Validate Custom Model

In [ ]:


!yolo task=detect mode=test model=/content/drive/MyDrive/p16-depth-estimation/runs/detect/train/weights/best.pt data=/content/drive/MyDrive/p16-depth-estimation/datasets/depth-estimation--4/data.yaml

Traceback (most recent call last):
  File "/usr/local/bin/yolo", line 8, in <module>
    sys.exit(entrypoint())
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/yolo/cfg/__init__.py", line 249, in entrypoint
    getattr(model, mode)(verbose=True, **overrides)
AttributeError: 'YOLO' object has no attribute 'test'. Did you mean: 'reset'?


## Inference with Custom Model

In [ ]:
%cd {HOME}
!yolo task=detect mode=predict model=/content/drive/MyDrive/p16-depth-estimation/runs/detect/train/weights/best.pt conf=0.05 source="/content/drive/MyDrive/Depth Estimation Project/photos/pics" save=True

[Errno 2] No such file or directory: '{HOME}'
/content
/bin/bash: yolo: command not found


**NOTE:** Let's take a look at few results.

In [ ]:
import glob
from IPython.display import Image, display

for image_path in glob.glob("/content/runs/detect/predict/*.jpg"):
      display(Image(filename=image_path, width=600))
      print("\n")

## Deploy model on Roboflow

Once you have finished training your YOLOv8 model, you’ll have a set of trained weights ready for use. These weights will be in the `/runs/detect/train/weights/best.pt` folder of your project. You can upload your model weights to Roboflow Deploy to use your trained weights on our infinitely scalable infrastructure.

The `.deploy()` function in the [Roboflow pip package](https://docs.roboflow.com/python) now supports uploading YOLOv8 weights.

To upload model weights, add the following code to the “Inference with Custom Model” section in the aforementioned notebook:

In [ ]:
project.version(dataset.version).deploy(model_type="yolov8", model_path=f"{HOME}/runs/detect/train/")

View the status of your deployment at: https://app.roboflow.com/p16-81bkd/depth-estimation/deploy/4
Share your model with the world at: https://universe.roboflow.com/p16-81bkd/depth-estimation/model/4


In [ ]:
#While your deployment is processing, checkout the deployment docs to take your model to most destinations https://docs.roboflow.com/inference

In [ ]:
#Run inference on your model on a persistant, auto-scaling, cloud API

#load model
model = project.version(dataset.version).model

#choose random test set image
import os, random
test_set_loc = dataset.location + "/test/images/"
random_test_image = random.choice(os.listdir(test_set_loc))
print("running inference on " + random_test_image)

pred = model.predict(test_set_loc + random_test_image, confidence=40, overlap=30).json()
pred

running inference on 25_13-06-2023_mp4-29_jpg.rf.4587040e4d2f6bedf47afe2c6f05ad15.jpg


HTTPError: ignored

In [ ]:
from ultralytics import YOLO
model = YOLO('/content/drive/MyDrive/p16-depth-estimation/runs/detect/train/weights/best.pt')
results = model.predict(source='/content/drive/MyDrive/Depth Estimation Project/data_depth/test/images/03_13-06-2023_mp4-19_jpg.rf.8f0041cc74695ab0784332664fe88a43.jpg', conf=0.25)

Ultralytics YOLOv8.0.20 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11127906 parameters, 0 gradients, 28.4 GFLOPs


In [ ]:
results

[Ultralytics YOLO <class 'ultralytics.yolo.engine.results.Boxes'> masks
 type: <class 'torch.Tensor'>
 shape: torch.Size([7, 6])
 dtype: torch.float32
  + tensor([[8.72000e+02, 6.37000e+02, 1.01300e+03, 7.61000e+02, 8.72634e-01, 1.00000e+00],
         [7.26000e+02, 6.27000e+02, 7.79000e+02, 7.74000e+02, 8.00209e-01, 1.00000e+00],
         [4.20000e+02, 6.19000e+02, 4.64000e+02, 7.64000e+02, 7.85085e-01, 2.00000e+00],
         [2.53000e+02, 6.44000e+02, 2.95000e+02, 8.09000e+02, 7.57707e-01, 2.00000e+00],
         [1.30000e+02, 6.54000e+02, 1.75000e+02, 8.24000e+02, 7.35159e-01, 2.00000e+00],
         [2.04000e+02, 6.53000e+02, 2.34000e+02, 8.10000e+02, 4.69588e-01, 2.00000e+00],
         [5.17000e+02, 6.63000e+02, 5.49000e+02, 7.31000e+02, 4.32924e-01, 2.00000e+00]], device='cuda:0')]

In [ ]:
from ultralytics import YOLO

# Load a pretrained YOLOv8n model
model = YOLO('/content/drive/MyDrive/p16-depth-estimation/runs/detect/train/weights/best.pt')

# Define path to video file
source = '/content/drive/MyDrive/Depth Estimation Project/Videos_without_Audio/10th July/100_10-07-2023.mp4'

# Run inference on the source
results = model(source, stream=True)  # generator of Results objects

Ultralytics YOLOv8.0.20 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11127906 parameters, 0 gradients, 28.4 GFLOPs


In [ ]:
!yolo task=detect mode=predict model=/content/drive/MyDrive/p16-depth-estimation/runs/detect/train/weights/best.pt conf=0.05 source="/content/drive/MyDrive/Depth Estimation Project/Videos_without_Audio/test" save=True

2023-08-18 07:56:42.580186: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-18 07:56:44.570078: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Ultralytics YOLOv8.0.20 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11127906 parameters, 0 gradients, 28.4 GFLOPs
video 1/1 (1/329) /content/drive/MyDrive/Depth Estimation Project/Videos_without_Audio/test/24_10-07-2023.mp4: 480x800 5 LMVs, 1 RoadDamages, 111.2ms
video 1/1 (2/329) /content/drive/MyDrive/Depth Estimation Project/Videos_without_Audio/test/24_10-07-2023.mp4: 480x800 5 LMVs, 16.2ms
video 1/1 (3/329) /content/drive/MyDrive/Depth Estimation Project/Videos_without_Audio/test/24_10-07-202

In [ ]:
result="/content/runs/detect/predict2/24_10-07-2023.mp4"

In [ ]:
result

'/content/runs/detect/predict2/24_10-07-2023.mp4'

In [ ]:
import cv2

In [ ]:
pip install timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.3 MB/s eta 0:00:00


In [ ]:
cv2.VideoCapture("/content/runs/detect/predict2/24_10-07-2023.mp4")

< cv2.VideoCapture 0x7f9b314f3c70>

In [ ]:
import cv2
import torch
import numpy as np
import time

# Path to the video file
video_path = "/content/runs/detect/predict2/24_10-07-2023.mp4"

# Model type to use
model_type = "MiDaS_small"

# Load the Midas model
midas = torch.hub.load("intel-isl/Midas", model_type)
midas_transforms = torch.hub.load("intel-isl/MiDaS", "transforms")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

midas = midas.to(device)
input_batch = input_batch.to(device)
# Choose the appropriate transform based on the model type
if model_type == "DPT_Large" or model_type == "DPT_Hybrid":
    transform = midas_transforms.dpt_transform
else:
    transform = midas_transforms.small_transform

cap = cv2.VideoCapture(video_path)
while cap.isOpened():
    success, img = cap.read()

    if not success:
        break

    start = time.time()
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    input_batch = transform(img).to(device)

    with torch.no_grad():
        prediction = midas(input_batch)

        prediction = torch.nn.functional.interpolate(
            prediction.unsqueeze(1),
            size=img.shape[:2],
            mode="bicubic",
            align_corners=False,
        ).squeeze()

    depth_map = prediction.cpu().numpy()
    print(depth_map[0])

  depth_map = cv2.normalize(depth_map, None, 0, 1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_64F)

    #end = time.time()
   # totalTime = end - start

    fps = 1 / totalTime

    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

  cv2.putText(img, f'FPS: {int(fps)}', (20, 70), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 2)
   # cv2.imshow("Image", img)
   #cv2.imshow("Depth Map", depth_map)
    #f cv2.waitKey(1) & 0xFF == ord('q'):
       break

cap.release()
cv2.destroyAllWindows()


Using cache found in /root/.cache/torch/hub/intel-isl_Midas_master
Using cache found in /root/.cache/torch/hub/rwightman_gen-efficientnet-pytorch_master


Loading weights:  None


Using cache found in /root/.cache/torch/hub/intel-isl_MiDaS_master


[     299.76      299.77      299.73 ...       195.6      194.73      194.95]
[     282.92      282.93      282.88 ...      178.93      178.12      178.32]
[     310.62      310.63      310.58 ...      216.86       215.9      216.14]
[     239.42      239.44      239.35 ...       168.6      167.83      168.02]
[     286.45      286.46      286.41 ...      180.67      179.91       180.1]
[     263.09      263.11      263.04 ...       157.5      156.81      156.99]
[      303.6       303.6      303.58 ...      158.27      157.61      157.78]
[      333.6      333.62      333.56 ...      184.05      183.21      183.42]
[     262.77      262.78      262.73 ...      194.05      193.22      193.42]
[     311.79      311.81      311.74 ...       251.3      250.21      250.48]
[     315.25      315.26      315.22 ...      173.09      172.31      172.51]
[        339      339.01      338.96 ...      237.78      236.78      237.03]
[     296.24      296.26      296.17 ...      164.18      163.46

In [ ]:
import cv2
import torch
import numpy as np
import time

# YOLO configuration and weights paths
config_path = "yolo_config.cfg"
weights_path = "yolo_weights.weights"

# Path to the image file
image_path = "C:\\Users\\HP\\Desktop\\DepthNet\\image.jpg"

# Model type to use for depth estimation
model_type = "MiDaS_small"

# Load YOLO model
net = cv2.dnn.readNetFromDarknet(config_path, weights_path)

# Load the Midas model
midas = torch.hub.load("intel-isl/Midas", model_type)
midas_transforms = torch.hub.load("intel-isl/MiDaS", "transforms")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Choose the appropriate transform based on the model type
if model_type == "DPT_Large" or model_type == "DPT_Hybrid":
    transform = midas_transforms.dpt_transform
else:
    transform = midas_transforms.small_transform

# Load the image
image = cv2.imread(image_path)

# Obtain the bounding box detections using YOLO
blob = cv2.dnn.blobFromImage(image, 1 / 255, (416, 416), swapRB=True, crop=False)
net.setInput(blob)
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
outs = net.forward(output_layers)

# Process each detected object
for detection in outs:
    for obj in detection:
        scores = obj[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]

        # Filter objects by confidence threshold
        if confidence > 0.5:
            center_x = int(obj[0] * image.shape[1])
            center_y = int(obj[1] * image.shape[0])
            width = int(obj[2] * image.shape[1])
            height = int(obj[3] * image.shape[0])

            # Extract object region of interest (ROI)
            x = int(center_x - width / 2)
            y = int(center_y - height / 2)
            roi = image[y:y + height, x:x + width]

            # Process the ROI for depth estimation
            start = time.time()
            roi = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
            input_batch = transform(roi).to(device)

            with torch.no_grad():
                prediction = midas(input_batch)

                prediction = torch.nn.functional.interpolate(
                    prediction.unsqueeze(1),
                    size=roi.shape[:2],
                    mode="bicubic",
                    align_corners=False,
                ).squeeze()

            depth_map = prediction.cpu().numpy()

            depth_map = cv2.normalize(depth_map, None, 0, 1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_64F)

            end = time.time()
            totalTime = end - start

            fps = 1 / totalTime

            depth_map = (depth_map * 255).astype(np.uint8)
            depth_map = cv2.applyColorMap(depth_map, cv2.COLORMAP_MAGMA)
            cv2.putText(roi, f'FPS: {int(fps)}', (20, 70), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 2)

            # Display the ROI with depth map
            cv2.imshow("ROI", roi)
            cv2.imshow("Depth Map", depth_map)
            cv2.waitKey(0)

cv2.destroyAllWindows()
